In [2]:
import os
import json
from collections import defaultdict
from tqdm import tqdm

OUT_DIR = '/app/data/aws_face_bboxes_new'

def get_video_name(s):
    return os.path.splitext(s.split('/')[-1])[0]

frame_3s_sampler = FrameSampler.objects.get(name='3s')
face_labeler = Labeler.objects.get(name='mtcnn:july-25-2019')

In [ ]:
if not os.path.exists(OUT_DIR):
    os.makedirs(OUT_DIR)

for video in tqdm(Video.objects.all().order_by('id')):
    video_name = get_video_name(video.path)
    frame_to_faces = defaultdict(list)
    for f in Face.objects.filter(
            frame__video=video, frame__sampler=frame_3s_sampler, labeler=face_labeler
        ).values('id', 'frame__number', 'bbox_x1', 'bbox_x2', 'bbox_y1', 'bbox_y2'):
        frame_to_faces[f['frame__number']].append({
            'id': f['id'], 'bbox': {
                'x1': f['bbox_x1'], 'x2': f['bbox_x2'], 'y1': f['bbox_y1'], 'y2': f['bbox_y2']
            }
        })
    video_out_file = os.path.join(OUT_DIR, video_name + '.faces.json')
    with open(video_out_file, 'w') as f:
        json.dump(list(frame_to_faces.items()), f)

 48%|█████████████████████████▍                           | 123581/257031 [49:38<19:34, 113.60it/s]

In [ ]:
video_and_fps = [(v.id, get_video_name(v.path), v.fps) for v in Video.objects.all()]

In [ ]:
with open('/app/data/videos_and_fps.json', 'w') as f:
    json.dump(video_and_fps, f)